# EfficientNet on Gaussian Blur Filter

We'll train EfficientNet on the Gaussian Blur filtered data

## Imports

In [ ]:
from keras.applications import EfficientNetB0
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import tensorflow as tf
import cv2


2023-11-02 23:01:24.353723: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-02 23:01:24.418412: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading Data

In [ ]:
train_data = np.load('Gaussian_Blur_Train.npz')
test_data = np.load('Gaussian_Blur_Test.npz')


In [ ]:
x_train = train_data['images']
y_train = train_data['labels']


In [ ]:
x_test = test_data['images']
y_test = test_data['labels']


In [ ]:
label_names = train_data['label_names']


In [ ]:
resized_x_train = []
for image in x_train:
    resized_image = cv2.resize(image, (224, 224))
    resized_x_train.append(resized_image)


In [ ]:
resized_x_test = []
for image in x_test:
    resized_image = cv2.resize(image, (224, 224))
    resized_x_test.append(resized_image)


In [ ]:
resized_x_train = np.array(resized_x_train)
resized_x_test = np.array(resized_x_test)
training_subset = resized_x_train


## Building the Model

In [ ]:
efficient_net = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


2023-11-02 23:01:51.667990: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22830 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:3d:00.0, compute capability: 7.5


In [ ]:
x = efficient_net.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
Ph = Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=efficient_net.input, outputs=Ph)


## Training the Model

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(training_subset, y_train, epochs=3, batch_size=32, validation_split=0.2)


Epoch 1/3


2023-11-02 23:02:21.988745: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2023-11-02 23:02:23.783558: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-11-02 23:02:23.963358: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-02 23:02:24.169746: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f0191266bb0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-02 23:02:24.169791: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Quadro RTX 6000, Compute Capability 7.5
2023-11-02 23:02:24.176239: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env

2500/2500 [==============================] - 339s 120ms/step - loss: 0.2223 - accuracy: 0.9107 - val_loss: 0.1435 - val_accuracy: 0.9471
Epoch 2/3
2500/2500 [==============================] - 297s 119ms/step - loss: 0.1534 - accuracy: 0.9400 - val_loss: 0.3344 - val_accuracy: 0.8982
Epoch 3/3
2500/2500 [==============================] - 300s 120ms/step - loss: 0.1274 - accuracy: 0.9512 - val_loss: 0.1350 - val_accuracy: 0.9492


In [ ]:
test_loss, test_accuracy = model.evaluate(resized_x_test, y_test)


625/625 [==============================] - 16s 26ms/step - loss: 0.1391 - accuracy: 0.9481


The model did really well, only about 3% worse than on the unfiltered images. This suggests that the Gaussian Blur doesn't change anything important about the images that our models could have been using to classify them